## Write to and read from a Delta Lake table

### Write a Spark DataFrame to a Delta Lake table

In [1]:
from pyspark.sql import SparkSession
import subprocess

spark = SparkSession.builder \
    .appName("DeltaLakeDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

delta_path = "/opt/spark/delta-tables/"

:: loading settings :: url = jar:file:/opt/spark-4.0.1-bin-hadoop3/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/spark/.ivy2.5.2/cache
The jars for the packages stored in: /home/spark/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.spark#spark-connect_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-538af058-1ec7-4fc0-b592-b4a45a2daca8;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.1 in central
	found io.delta#delta-storage;4.0.1 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	found org.apache.spark#spark-connect_2.13;4.1.1 in central
	found org.apache.spark#spark-pipelines_2.13;4.1.1 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0 in central
	found jakarta.servlet#jakarta.servlet-api;5.0.0 in central
	found javax.servlet#javax.servlet-api;4.0.1 in central
	found com.google.guava#guava;33.4.8-jre in central
	fo

In [2]:
data = spark.range(0, 5)

(data
  .write
  .format("delta")
  .mode("overwrite")
  .save(delta_path + "table1")
)

### Read the above Delta Lake table to a Spark DataFrame and display the DataFrame

In [3]:
df = (spark
        .read
        .format("delta")
        .load(delta_path + "table1")
        .orderBy("id")
      )

df.show()

26/02/18 03:54:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



## Overwrite a Delta Lake table

### Overwrite the Delta Lake table written in the above step

In [4]:
data = spark.range(5, 10)

view_data = data.createOrReplaceTempView("data_view")
table2 = delta_path + "table1"
spark.sql(f"""
  MERGE INTO delta.`{table2}` AS target
  USING data_view AS source
  ON target.id = source.id
  WHEN NOT MATCHED THEN
    INSERT *
""")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

### Read the above overwritten Delta Lake table to a Spark DataFrame and display the DataFrame

In [5]:
df = (spark
        .read
        .format("delta")
        .load(delta_path + "table1")
        .orderBy("id")
      )

df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



## Delta Lake and [ACID](https://en.wikipedia.org/wiki/ACID)

### Update Delta Lake Table

In [6]:
from delta.tables import *
from pyspark.sql.functions import *


delta_table = DeltaTable.forPath(spark, delta_path + "table1")

spark.sql(f"""
  UPDATE delta.`{delta_path}table1`
  SET id = id + 100
  WHERE id % 2 == 0
""")


26/02/18 03:54:30 WARN UpdateCommand: Could not validate number of records due to missing statistics.


DataFrame[num_affected_rows: bigint]

In [7]:
spark.sql("""
vacuum delta.`/opt/spark/delta-tables/table1`
""")

Deleted 0 files and directories in a total of 1 directories.


DataFrame[path: string]

In [ ]:
spark.sql("""
        optimize delta.`/opt/spark/delta-tables/table1`
        """)

In [11]:
spark.sql(f"""
    ALTER TABLE delta.`{delta_path}table1`
    SET TBLPROPERTIES (
        'delta.enableChangeDataCapture' = 'true'
    )
""")

DataFrame[]

In [15]:
delta_table.toDF().orderBy("id").show()
delta_table.toDF().createOrReplaceTempView("delta_table_view")
spark.sql("""
  select * from detla_table_view
  order by id
""").show()

{"ts": "2026-02-18 03:56:50.010", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `detla_table_view` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01", "context": {"errorClass": "TABLE_OR_VIEW_NOT_FOUND"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o35.sql.\n: org.apache.spark.sql.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `detla_table_view` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXIST

+---+
| id|
+---+
|  1|
|  3|
|  5|
|  7|
|  9|
+---+



AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `detla_table_view` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 2 pos 16;
'Sort ['id ASC NULLS FIRST], true
+- 'Project [*]
   +- 'UnresolvedRelation [detla_table_view], [], false


In [ ]:
spark.sql(f"""
  UPDATE delta.`{delta_path}table2`
  SET id = id + 100
  WHERE id % 2 == 0
""")
delta_table2.toDF().createOrReplaceTempView("delta_table2_view")


AnalysisException: [PATH_NOT_FOUND] Path does not exist: /opt/spark/delta-tables/table2. SQLSTATE: 42K03

###  `delete`

In [14]:
# Delete every even value
(delta_table
  .delete(
    condition = expr("id % 2 == 0")
  )
)

(delta_table
  .toDF()
  .orderBy("id")
  .show()
)

26/02/18 03:56:09 WARN DeleteCommand: Could not validate number of records due to missing statistics.


+---+
| id|
+---+
|  1|
|  3|
|  5|
|  7|
|  9|
+---+



### `merge` Delta Lake Table

In [ ]:
# Upsert (merge) new data
new_data = spark.range(0, 20)

(delta_table.alias("old_data")
  .merge(
      new_data.alias("new_data"),
      "old_data.id = new_data.id"
      )
  .whenMatchedUpdate(set = { "id": col("new_data.id") })
  .whenNotMatchedInsert(values = { "id": col("new_data.id") })
  .execute()
)

(delta_table
  .toDF()
  .orderBy("id")
  .show()
)

## Time travel feature of Delta Lake

### Display the entire history of the above Delta Lake table

In [ ]:
# get the full history of the table
delta_table_history = (DeltaTable
                        .forPath(spark, f"{delta_path}delta-table")
                        .history()
                      )

(delta_table_history
   .select("version", "timestamp", "operation", "operationParameters", "operationMetrics", "engineInfo")
   .show()
)

### Latest version of the Delta Lake table

In [ ]:
# get the full history of the table
delta_table_history = (DeltaTable
                        .forPath(spark, "/tmp/delta-table")
                        .history()
                      )

(delta_table_history
   .select("version", "timestamp", "operation", "operationParameters", "operationMetrics", "engineInfo")
   .show()
)

### Latest version of the Delta Lake table

In [ ]:
df = (spark
        .read
        .format("delta")
        .load("/tmp/delta-table")
        .orderBy("id")
      )

df.show()

### Time travel to the version `0` of the Delta Lake table using Delta Lake's history feature

In [ ]:
df = (spark
        .read
        .format("delta")
        .option("versionAsOf", 0) # we pass an option `versionAsOf` with the required version number we are interested in
        .load("/tmp/delta-table")
        .orderBy("id")
      )

df.show()

### Time travel to the version `3` of the Delta Lake table using Delta Lake's  history feature

In [ ]:
df = (spark
        .read
        .format("delta")
        .option("versionAsOf", 3) # we pass an option `versionAsOf` with the required version number we are interested in
        .load("/tmp/delta-table")
        .orderBy("id")
      )

df.show()